## Pipetting scheduling optimization with CVRP

In [7]:
import numpy as np
from utils import random_task_generation, print_command, get_optimized_sequence
from pipette_scheduler import CVRP_pipette_scheduling, calculate_X,calculate_D,calculate_D_prime,calculate_S_E
from ortools_solver import CVRP_solver

## Input Task Matrix
### Option 1: Import from CSV file
Load your task matrix from a CSV file:
- **Dimensions**: n×m matrix where n and m should be standard well plate sizes (12, 24, 96, or 384)
- **Values**: Non-zero elements represent the volume of liquid transfer (in µL)
- **Zero elements**: Indicate no transfer required between those wells
- **Format**: Rows represent source wells, columns represent destination wells

In [ ]:
task_matrix = np.genfromtxt('data/task_matrix.csv', delimiter=',')
transfer_list = np.argwhere(task_matrix)

### Option 2: Use random generation (for testing)

In [2]:
np.random.seed(0)
num_candidates = 100
random_task_matrix = random_task_generation(12,12,num_candidates)
random_transfer_list = np.argwhere(random_task_matrix)
volumes = random_task_matrix[(random_transfer_list[:, 0], random_transfer_list[:, 1])]

# Run optimization
Use `CVRP_pipette_scheduling` to optimize the pipetting task sequence. This function applies the Capacitated Vehicle Routing Problem (CVRP) algorithm to minimize the total travel distance.

### Parameters:
- **`task_matrix`**: Input matrix (n×m) with transfer volumes in µL
- **`aspirate_t`**: Fixed time (seconds) for aspiration operations - moving tips down/up and arm movement (default: 1 second)
- **`aspirate_speed`**: Aspiration speed in µL/s (default: 10)
- **`dispense_t`**: Fixed time (seconds) for dispensing operations - moving tips down/up and arm movement (default: 1 second)
- **`dispense_speed`**: Dispensing speed in µL/s (default: 10)
- **`cvrp_timewall`**: Maximum solving time in seconds for the CVRP solver (default: 10 second)
- **`decimal_points`**: Number of decimal points for numerical precision (default: 2)

### Returns:
- **`optimized_distance`**: Optimized total travel time
- **`optimized_sequence`**: Optimized sequence of pipetting operations

In [3]:
cvrp_distance,cvrp_sequence = CVRP_pipette_scheduling(random_task_matrix, 2, 30, 1, 20)

## Process Optimization Results

After obtaining the optimized sequence from CVRP, we need to post-process the results to extract the actual pipetting operations in a usable format.

### What this code does:
1. **Flatten the sequence**: Convert the 2D CVRP sequence array to a 1D array
2. **Remove padding**: Filter out -1 values (used as padding in CVRP solver)
3. **Adjust indexing**: Convert from 1-based (CVRP solver) to 0-based (Python) indexing
4. **Map to transfers**: Use the processed indices to get actual source→destination well pairs
5. **Extract volumes**: Get the corresponding transfer volumes for each operation

### Output variables:
- **`output_sequence`**: Array of [source_well, destination_well] pairs in optimized order
- **`output_volumes`**: Array of transfer volumes corresponding to each operation

### Example output format:
```
output_sequence: [[2, 5], [7, 3], [1, 8], ...]  # Well pairs
output_volumes:  [15.3, 22.1, 8.7, ...]         # Volumes in µL
```

This gives you the complete optimized pipetting protocol ready for exporting to your liquid handler.

In [4]:
flatten_cvrp_sequence = cvrp_sequence.flatten()
flatten_cvrp_sequence = flatten_cvrp_sequence[flatten_cvrp_sequence!=-1] -1
flatten_cvrp_sequence =flatten_cvrp_sequence.astype(int)
output_sequence = random_transfer_list[flatten_cvrp_sequence]
output_volumes = volumes[flatten_cvrp_sequence]

## Generate Worklist Commands

You can also use our `print_command` function for generating a formatted worklist that can be exported to liquid handling systems.


In [5]:
commands = print_command(flatten_cvrp_sequence,random_transfer_list,f'source_name', f'destination_name',volumes)
commands

array([['source_name', '10', 'destination_name', '4', '12.0'],
       ['source_name', '11', 'destination_name', '5', '24.0'],
       ['source_name', '12', 'destination_name', '6', '29.0'],
       ['source_name', '4', 'destination_name', '10', '24.0'],
       ['source_name', '5', 'destination_name', '6', '6.0'],
       ['source_name', '10', 'destination_name', '7', '62.0'],
       ['source_name', '11', 'destination_name', '8', '60.0'],
       ['source_name', '12', 'destination_name', '9', '85.0'],
       ['source_name', '1', 'destination_name', '10', '68.0'],
       ['source_name', '2', 'destination_name', '4', '30.0'],
       ['source_name', '3', 'destination_name', '5', '35.0'],
       ['source_name', '2', 'destination_name', '6', '37.0'],
       ['source_name', '3', 'destination_name', '4', '81.0'],
       ['source_name', '7', 'destination_name', '5', '83.0'],
       ['source_name', '8', 'destination_name', '6', '86.0'],
       ['source_name', '9', 'destination_name', '10', '19.0'],


Alternatively, the distance could be calculated as formula 5a

In [ ]:
nwells_source, nwells_destination = random_task_matrix.shape
D_S = calculate_D(nwells_source)
D_D = calculate_D(nwells_destination)
S, E, volumes = calculate_S_E(random_task_matrix)
D_prime = calculate_D_prime(D_S, D_D, S, E, volumes, 1, 100, 1, 100)
# scale D_prime to avoid numerical issues
_, recorder = CVRP_solver(np.round(D_prime* 100).astype(np.int64), solving_time=10)
cvrp_sequence = get_optimized_sequence(recorder)
X = calculate_X(cvrp_sequence)
cvrp_distance = np.trace(np.dot(X.T, np.round(D_prime * 100))) / 100    # formula 5as
print("CVRP distance: ", cvrp_distance)

CVRP distance:  154.59
